In [ ]:
from poppy.creatures import PoppyHumanoid
import math
import time
import numpy as np

In [2]:
poppy = PoppyHumanoid()

In [3]:
def get_Jacobian(theta):
    ARM_LENGTH_1 = 15.1
    ARM_LENGTH_2 = 10.1
    return np.array([
                [-np.sin(theta[0])*ARM_LENGTH_1 - ARM_LENGTH_2*np.sin(theta[0]+theta[1]), 
                -ARM_LENGTH_2*np.sin(theta[0]+theta[1])], 

                [np.cos(theta[0])*ARM_LENGTH_1 + ARM_LENGTH_2*np.cos(theta[0]+theta[1]), 
                ARM_LENGTH_2*np.cos(theta[0]+theta[1])]
            ])

def get_control(distance, theta):
    J = get_Jacobian(theta)
    u = np.dot(np.linalg.pinv(J), distance)
    return u

def get_left_end_effector_position(theta):
    ARM_LENGTH_1 = 15.1
    ARM_LENGTH_2 = 10.1
    OFFSET = 0.0 #10.95
    pos = np.array([0.0, 0.0])
    pos[0] = ARM_LENGTH_1*np.cos(theta[0]) + ARM_LENGTH_2*np.cos(theta[0]+theta[1])
    pos[1] = OFFSET + ARM_LENGTH_1*np.sin(theta[0]) + ARM_LENGTH_2*np.sin(theta[0]+theta[1])
    return pos

def set_action(action):
    if action==1:
        poppy.l_shoulder_x.goto_position(poppy.l_shoulder_x.present_position - 1, 0, wait=True)
    elif action==2:
        poppy.l_shoulder_x.goto_position(poppy.l_shoulder_x.present_position + 1, 0, wait=True) 
    elif action==3:
        poppy.l_elbow_y.goto_position(poppy.l_elbow_y.present_position - 1, 0, wait=True)
    elif action==4:
        poppy.l_elbow_y.goto_position(poppy.l_elbow_y.present_position + 1, 0, wait=True)
    return

In [5]:
l_motors = [poppy.l_shoulder_y, poppy.l_shoulder_x, poppy.l_arm_z, poppy.l_elbow_y]
r_motors = [poppy.r_shoulder_y, poppy.r_shoulder_x, poppy.r_arm_z, poppy.r_elbow_y]
for m in l_motors+r_motors:
    m.compliant = False

# left init
poppy.l_shoulder_y.goto_position(-90,1,wait=True)
poppy.l_arm_z.goto_position(-90,1,wait=True)

# right init
poppy.r_shoulder_y.goto_position(-90,1,wait=True)
poppy.r_arm_z.goto_position(90,1,wait=True)

# some starting position
poppy.l_elbow_y.goto_position(-90,1,wait=True)
poppy.l_shoulder_x.goto_position(60, 1, wait=True)

theta = [poppy.l_shoulder_x.present_position*np.pi/180.0, poppy.l_elbow_y.present_position*np.pi/180.0]
hand = get_left_end_effector_position(theta)
goal = np.array([25., 0.])

print hand, goal

[ 23.13621867   5.42693652] [ 25.   0.]


In [15]:
for i in range(300):
    theta = [poppy.l_shoulder_x.present_position*np.pi/180.0, poppy.l_elbow_y.present_position*np.pi/180.0]
    hand = get_left_end_effector_position(theta)
    distance = goal - hand
    print i, theta, goal, hand, np.linalg.norm(distance)
    
    if np.linalg.norm(distance) < 1.0:
        break
    
    u = get_control(distance, theta)
    
    waiting = False
    if np.argmax(np.abs(u)) == 0:
        if u[0] < 0:
            poppy.l_shoulder_x.goto_position(poppy.l_shoulder_x.present_position - 1, 0, wait=waiting)
        else:
            poppy.l_shoulder_x.goto_position(poppy.l_shoulder_x.present_position + 1, 0, wait=waiting) 
    else:
        if u[1] < 0:
            poppy.l_elbow_y.goto_position(poppy.l_elbow_y.present_position - 1, 0, wait=waiting)
        else:
            poppy.l_elbow_y.goto_position(poppy.l_elbow_y.present_position + 1, 0, wait=waiting)
    
    time.sleep(0.1)

0 [-0.027227136331111578, 0.0054105206811824215] [ 25.   0.] [ 25.19199988  -0.6314093 ] 0.659955801938
1 [-0.025656340004316623, 0.011519173063162576] [ 25.   0.] [ 25.19402123  -0.53014886] 0.564537026578
2 [-0.027227136331111578, 0.01308996938995747] [ 25.   0.] [ 25.19339413  -0.55385959] 0.586652997352
3 [-0.025656340004316623, 0.01308996938995747] [ 25.   0.] [ 25.19423305  -0.51428524] 0.549741558618
4 [-0.027227136331111578, 0.01308996938995747] [ 25.   0.] [ 25.19339413  -0.55385959] 0.586652997352
5 [-0.027227136331111578, 0.01308996938995747] [ 25.   0.] [ 25.19339413  -0.55385959] 0.586652997352
6 [-0.025656340004316623, 0.01308996938995747] [ 25.   0.] [ 25.19423305  -0.51428524] 0.549741558618
7 [-0.025656340004316623, 0.01308996938995747] [ 25.   0.] [ 25.19423305  -0.51428524] 0.549741558618
8 [-0.027227136331111578, 0.01308996938995747] [ 25.   0.] [ 25.19339413  -0.55385959] 0.586652997352
9 [-0.027227136331111578, 0.01308996938995747] [ 25.   0.] [ 25.19339413  -0.55

In [ ]:
poppy.close()